# Analyse BlueSeis BSPF Event Analysis - Polarization

With pressure sensor parascientific and new sensor 

In [1]:
import os, sys
import obspy as obs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy

sys.path.append("/home/andbro/andbro_python")

from andbro__store_as_pickle import __store_as_pickle
from andbro__savefig import __savefig

In [ ]:
from functions.request_data import __request_data

In [ ]:
sys.path.append("/home/andbro/Documents/ROMY/twistpy/TwistPy/")

from twistpy.utils import stransform
from twistpy.convenience import ricker
from twistpy.polarization import (
    TimeDomainAnalysis6C,
    PolarizationModel6C,
    SupportVectorMachine,
    EstimatorConfiguration,
)

In [ ]:
if os.uname().nodename == 'lighthouse':
    root_path = '/home/andbro/'
    data_path = '/home/andbro/kilauea-data/'
    archive_path = '/home/andbro/freenas/'
elif os.uname().nodename == 'kilauea':
    root_path = '/home/brotzer/'
    data_path = '/import/kilauea-data/'
    archive_path = '/import/freenas-ffb-01-data/'

## Configurations

In [ ]:
config = {}

## BSPF coordinates
config['BSPF_lon'] = -116.455439
config['BSPF_lat'] = 33.610643

##
config['path_to_mseed'] = data_path+"BSPF/data/waveforms/ACC/"

## event selction
config['path_to_data'] = data_path+"BSPF/data/"

## output path for figures
config['output_figs'] = data_path+"BSPF/figures/velocity/"


## Load Data

In [ ]:
events = obs.read_events(config['path_to_data']+"catalogs/BSPF_catalog_20221001_20230615_all.xml")

triggered = pd.read_pickle(config['path_to_data']+"catalogs/BSPF_catalog_20221001_20230615_triggered.pkl")

event_times = pd.read_pickle(config['path_to_data']+"event_selection_good.pkl")

In [ ]:
num = 5

event_time = event_times.origin_time.loc[num]

event = events.filter(f"time >= {event_time}",f"time <= {event_time}")

trig = triggered[triggered.origin == event_time]

## prepare string
event_time_str = event_time.split(".")[0].replace(" ","_").replace("-","").replace(":","")

In [ ]:
st = obs.read(config['path_to_mseed']+f"*{event_time_str}*")

st.resample(40).trim(st[0].stats.starttime, st[0].stats.endtime)

st.detrend("simple")

# st.plot(equal_scale=False);

In [ ]:
# cut_offs = [
#     (65,80), (65,80), (60,100), (60,100), (60,70),
#     (60,90), (60,90), (60,90), (62,105), (65,90),
#     (62,110), (65,85), (60,105),
# ]

# st.trim(st[0].stats.starttime+cut_offs[num][0]-5, st[0].stats.endtime-cut_offs[num][1]+10);

## S-Transform

In [ ]:
from functions.makeplotStream_cwt import __makeplotStream_cwt

In [ ]:
fig = __makeplotStream_cwt(st_all, config);

## Frequency Domain

In [ ]:
from twistpy.polarization import TimeFrequencyAnalysis6C, EstimatorConfiguration

In [ ]:
acc = ii_pfo_acc.copy()
rot = py_bspf.copy()

acc.resample(20.0)
rot.resample(20.0)

scaling_velocity = 5000 # m/s

for tr1, tr2 in zip(acc, rot):
    tr1.taper(0.1);  tr2.taper(0.1)
    tr1.detrend("linear") ; tr2.detrend("linear")
    
for tr in acc: 
    tr.data /= scaling_velocity


window = {"number_of_periods": 2, "frequency_extent": 0.01}

In [ ]:
acc.plot(), rot.plot()

In [ ]:
%matplotlib inline


for tr in st_all:
    plt.plot(tr.data)
plt.show()

In [ ]:
analysis = TimeFrequencyAnalysis6C(
                                    traN=acc.select(channel="*N")[0],
                                    traE=acc.select(channel="*E")[0],
                                    traZ=acc.select(channel="*Z")[0],
                                    rotN=rot.select(channel="*N")[0],
                                    rotE=rot.select(channel="*E")[0],
                                    rotZ=rot.select(channel="*Z")[0],
                                    window=window,
                                    dsfacf=100,
                                    dsfact=1,
                                    frange=[0.01, 15.0],
                                    scaling_velocity=scaling_velocity,
)

In [ ]:
est = EstimatorConfiguration(
                            wave_types=["R"],
                            method="DOT",
                            scaling_velocity=scaling_velocity,
                            use_ml_classification=False,
                            vr=[2000, 4000, 100],
                            xi=[-90, 90, 2],
                            phi=[0, 360, 10],
                            eigenvector=0,

)

In [ ]:
analysis.polarization_analysis(estimator_configuration=est)

In [ ]:
analysis.plot_wave_parameters(estimator_configuration=est, lh_min=0.5, lh_max=1.0)

## Polarization Analysis

In [ ]:
dt = py_bspf[0].stats.delta

scaling_velocity = 5000.0

### Train Model

In [ ]:
svm = SupportVectorMachine(name="love_wave_test2")

########################################################################################################################
# Now we can train the model. For details, please check the example on how to train a
# 'twistpy.machinelearning.SupportVectorMachine' object. In short, we want to train the model for wave parameters
# that are typical for the near surface, and we want to be able to identify P, SV, SH and Rayleigh waves. This means
# that we do not make a distinction between Love and SH waves here, and Love waves will simply be contained in the SH
# wave class as the special case of horizontally propagating SH waves. Additionally, we make use of a Noise class, for
# analysis windows with a random polarization. We allow waves to arrive from all directions (azimuth range [0 360]
# degrees and inclination range [0 90] degrees).

svm.train(
            wave_types=["R", "P", "SV", "L", "Noise"],
            N=5000,
            scaling_velocity=scaling_velocity,
            vp=(2000, 4000),
            vp_to_vs=(1.7, 2.4),
            vl=(100, 3000),
            vr=(100, 3000),
            phi=(0, 360),
            theta=(0, 90),
            xi=(-90, 90),
            free_surface=True,
            C=1,
            kernel="rbf",
)

## Prepare

In [ ]:
rot = py_bspf.copy().trim(config['tbeg'], config['tend']).taper(0.1)
acc = ii_pfo_acc.copy().trim(config['tbeg'], config['tend']).taper(0.1)

In [ ]:
acc.select(channel="*Z")[0].data *= -1
rot.select(channel="*Z")[0].data *= -1

In [ ]:
for tr in acc:
    tr.data /= scaling_velocity

In [ ]:
# acc.plot();
# rot.plot();

## Time Domain

In [ ]:
acc[0].stats.

In [ ]:
########################################################################################################################
# Now that we have trained the model, we can set up our analysis. We will perform 6C polarization analysis in the time
# domain and use a sliding time window that is 0.05 s long (50 samples) with an overlap between subsequent windows of
# 50%.

dt = py_bspf[0].stats.delta

window = {"window_length_seconds": 5 * dt, "overlap": 50}

analysis = TimeDomainAnalysis6C(
                                traN=acc.select(channel="*N")[0],
                                traE=acc.select(channel="*E")[0],
                                traZ=acc.select(channel="*Z")[0],
                                rotN=rot.select(channel="*N")[0],
                                rotE=rot.select(channel="*E")[0],
                                rotZ=rot.select(channel="*Z")[0],
                                window=window,
                                scaling_velocity=scaling_velocity,
)

In [ ]:
%matplotlib inline

In [ ]:
########################################################################################################################
# To classify the waves, we simply do (yielding a classification of the first eigenvector of the covariance matrix):

analysis.classify(svm=svm, eigenvector_to_classify=0)

classification = analysis.classification["0"]

t_windows = (analysis.t_windows)  # Positions of the sliding time windows where the classification was performed

#  Wave parameter estimation

# est = EstimatorConfiguration(
#                             wave_types=["L", "R"],
#                             method="DOT",
#                             scaling_velocity=scaling_velocity,
#                             use_ml_classification=False,
    
#                             svm=svm,
# )

est = EstimatorConfiguration(
                            wave_types=["R"],
                            method="DOT",
                            scaling_velocity=scaling_velocity,
                            use_ml_classification=False,
                            vr=[1000, 4500, 100],
                            xi=[-90, 90, 2],
                            phi=[0, 360, 10],
                            eigenvector=0,
                            svm=svm,
)


analysis.polarization_analysis(estimator_configuration=est)

analysis.plot(estimator_configuration=est, dop_clip=0.9)


In [ ]:
time domain grid search für parameter



## Test Example

In [ ]:
# data = obs.read("/home/andbro/Documents/ROMY/twistpy/TwistPy/example_data/ROMY_gulf_of_alaska_teleseism.mseed")

# # scaling_velocity = 4500.0

# for n, trace in enumerate(data):
#     trace.detrend("spline", order=5, dspline=100)
#     trace.trim(starttime=trace.stats.starttime, endtime=trace.stats.endtime - 4500)
#     trace.taper(0.2)
#     if n < 3:
#         trace.data /= scaling_velocity

# window = {"number_of_periods": 1, "frequency_extent": 0.01}

# analysis = TimeFrequencyAnalysis6C(
#     traN=data[0],
#     traE=data[1],
#     traZ=data[2],
#     rotN=data[3],
#     rotE=data[4],
#     rotZ=data[5],
#     window=window,
#     dsfacf=20,
#     dsfact=20,
#     frange=[0.01, 0.15],
# )

# est = EstimatorConfiguration(
#     wave_types=["R"],
#     method="DOT",
#     scaling_velocity=scaling_velocity,
#     use_ml_classification=False,
#     vr=[3000, 4000, 100],
#     xi=[-90, 90, 2],
#     phi=[150, 210, 1],
#     eigenvector=0,
# )

# analysis.plot_wave_parameters(estimator_configuration=est, lh_min=0.7, lh_max=1.0)
